In [8]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import tensorflow as tf
import numpy as np

df = pd.read_csv("data/daily-min-temperatures.csv")
data = df.filter(['Temp']).values

# Split data into training and testing sets
training_data_len = int(len(data) * 0.8)
train_data = data[0:training_data_len, :]
scaler = MinMaxScaler(feature_range=(0,1))
train_data = scaler.fit_transform(train_data)

x_train = []
y_train = []

for i in range(7, len(train_data)):
    x_train.append(train_data[i-7:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Define LSTM model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(tf.keras.layers.LSTM(128, return_sequences=False))
model.add(tf.keras.layers.Dense(1))

# Training
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=256, epochs=100)

test_data = data[training_data_len - 7:, :]
scaler = MinMaxScaler(feature_range=(0,1))
test_data = scaler.fit_transform(test_data)

x_test = []
y_test = data[training_data_len:, :]

for i in range(7, len(test_data)):
    x_test.append(test_data[i-7:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

future_data = df.filter(['Temp'])
last_data = future_data[-7:].values
last_data_scaled = scaler.transform(last_data)
X_test = []
X_test.append(last_data_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
pred = model.predict(X_test)
pred = scaler.inverse_transform(pred)
print('Predicted temp for the next day:', pred[0][0])

Epoch 1/100
12/12 [==============================] - 2s 19ms/step - loss: 0.0725 - accuracy: 6.8658e-04
Epoch 2/100
12/12 [==============================] - 0s 26ms/step - loss: 0.0171 - accuracy: 6.8658e-04
Epoch 3/100
12/12 [==============================] - 0s 26ms/step - loss: 0.0135 - accuracy: 0.0010
Epoch 4/100
12/12 [==============================] - 0s 28ms/step - loss: 0.0127 - accuracy: 0.0010
Epoch 5/100
12/12 [==============================] - 0s 27ms/step - loss: 0.0125 - accuracy: 0.0010
Epoch 6/100
12/12 [==============================] - 0s 32ms/step - loss: 0.0122 - accuracy: 0.0010
Epoch 7/100
12/12 [==============================] - 0s 27ms/step - loss: 0.0121 - accuracy: 0.0010
Epoch 8/100
12/12 [==============================] - 0s 27ms/step - loss: 0.0121 - accuracy: 0.0010
Epoch 9/100
12/12 [==============================] - 0s 27ms/step - loss: 0.0120 - accuracy: 0.0010
Epoch 10/100
12/12 [==============================] - 0s 28ms/step - loss: 0.0120 - accuracy